In [1]:
import librosa
import os

from IPython import display
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

from __future__ import division
import librosa.display
import maad
from soundsig.sound import BioSound
import warnings
warnings.simplefilter('ignore')
from audiomentations import Compose, AddGaussianSNR, TimeMask
import pickle

In [2]:
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all

,file,source,category,weight,fold
0,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,0
1,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
2,../data/interim/GoogleAudioSet_unbalanced_list...,Google_nature,0,1,5
3,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
4,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
...,...,...,...,...,...
13662,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,5
13663,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,3
13664,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
13665,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8


In [3]:
def feature_aug(y, fs):
    
    augment = Compose([
        AddGaussianSNR(min_snr_in_db=3.0,max_snr_in_db=12.0,p=1.0)
        ])

    y = augment(samples=y, sample_rate=fs)
    
   
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]

    return y, df_indices, mps, wt, wf


def audio_aug(filename):
    
    # open a file, where you stored the pickled data
    file = open(filename, 'rb')

    # dump information to that file
    output = pickle.load(file)
    y_aug, df_indices_aug, mps_aug, wt, wf = feature_aug(output['y'], 16000)
    y_bg_aug, df_indices_bg_aug, mps_bg_aug, wt, wf  = feature_aug(output['bg_y'], 16000)
    y_fg_aug, df_indices_fg_aug, mps_fg_aug, wt, wf  = feature_aug(output['fg_y'], 16000)

    # close the file
    file.close()
    
    output = {'y_aug': y_aug, 'y_bg_aug': y_bg_aug, 'y_fg_aug': y_fg_aug,
         'df_indices_aug': df_indices_aug, 'mps_aug': mps_aug, 'wt': wt, 'wf': wf, 
         'df_indices_fg_aug': df_indices_fg_aug, 'mps_fg_aug': mps_fg_aug,
         'df_indices_bg_aug': df_indices_bg_aug, 'mps_bg_aug': mps_bg_aug}
    
    save_file_name = '../data/interim/augmented_'+filename[16:-6]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
    f = open(save_file_name,'wb') # create a binary pickle file 
    pickle.dump(output,f)
    f.close()
    
    return



In [4]:
from joblib import Parallel, delayed
Parallel(n_jobs=-1)(delayed(audio_aug)(row['file']) for index, row in df_all.iterrows())

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

...

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/maad/util/miscellaneous.py:627: RuntimeWarning: Mean of empty slice.
  e_mean = e.mean(axis)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/numpy/core/_methods